In [1]:
import scipy.sparse as spr
import sys
path2oti = '../../../build/'
sys.path.append(path2oti) # Add path to OTI library.

import pyoti.real   as r
import pyoti.sparse as oti 
import pyoti.core   as coti
import pyoti.fem    as fem 

import pyoti.static.onumm1n1   as dual





%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

e  = oti.e
np = oti.np


In [2]:
fem.set_global_algebra(dual)
alg = fem.get_global_algebra()

In [3]:
Th = fem.line(0,1,0.005)
print(Th)

< mesh (pyoti.static.onumm1n1) object with 201 nodes, 202 elements of types: point1 (2), line2 (200) >


/Users/maristi7/opt/anaconda3/envs/pyoti/lib/python3.7/site-packages/numpy/ctypeslib.py:523: RuntimeWarning: A builtin ctypes object gave a PEP3118 format string that does not match its itemsize, so a best-guess will be made of the data type. Newer versions of python may behave correctly.
  return array(obj, copy=False)


In [3]:
import pyvista as pv
p = pv.BackgroundPlotter()

grid = Th.to_pv()

p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=16,cmap='jet')

< mesh (pyoti.real) object with 201 nodes, 202 elements of types: point1 (2), line2 (200) >


/Users/maristi7/opt/anaconda3/envs/pyoti/lib/python3.7/site-packages/numpy/ctypeslib.py:523: RuntimeWarning: A builtin ctypes object gave a PEP3118 format string that does not match its itemsize, so a best-guess will be made of the data type. Newer versions of python may behave correctly.
  return array(obj, copy=False)


In [5]:
# The problem to be solved:
# 
# 
Th.elements[1]


{'types': array([1], dtype=int32),
 'tags': [array([  3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,
          16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,
          29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
          42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,
          55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,
          68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,
          81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,
          94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106,
         107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119,
         120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132,
         133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145,
         146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158,
         159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 

In [8]:
Th.elements[0]

{'types': array([15], dtype=int32),
 'tags': [array([1, 2], dtype=uint64)],
 'indices': [array([[0],
         [1]], dtype=uint64)]}

In [9]:
x = oti.elm_help()
x

< elm_help object: 
 - Object allocation: -------------- Not allocated
end elm_help object >

In [6]:
elLine = fem.element[1]
elLine.end()
elLine.allocate(4)
elLine.allocate_spatial(1,compute_Jinv = True)


In [7]:
elLine.N

feomatm1n1< nip: 4, 
(Integration point - 0) 
-------------------------
omatm1n1< shape: (1, 2), 
 - Column 0
(0,0) 0.9306 + 0.0000 * e([1])
 - Column 1
(0,1) 0.0694 + 0.0000 * e([1])
>
-------------------------
(Integration point - 1) 
-------------------------
omatm1n1< shape: (1, 2), 
 - Column 0
(0,0) 0.0694 + 0.0000 * e([1])
 - Column 1
(0,1) 0.9306 + 0.0000 * e([1])
>
-------------------------
(Integration point - 2) 
-------------------------
omatm1n1< shape: (1, 2), 
 - Column 0
(0,0) 0.6700 + 0.0000 * e([1])
 - Column 1
(0,1) 0.3300 + 0.0000 * e([1])
>
-------------------------
(Integration point - 3) 
-------------------------
omatm1n1< shape: (1, 2), 
 - Column 0
(0,0) 0.3300 + 0.0000 * e([1])
 - Column 1
(0,1) 0.6700 + 0.0000 * e([1])
>
-------------------------
>

In [8]:
elLine.set_coordinates(Th.x,Th.y,Th.z,Th.elements[1]['indices'][0][199])

In [9]:
elLine.compute_jacobian()

In [10]:
alg.gauss_integrate(-alg.dot(alg.transpose(elLine.Nx),elLine.Nx)+alg.dot(alg.transpose(elLine.N),elLine.N),elLine.dV)

omatm1n1< shape: (2, 2), 
 - Column 0
(0,0) -199.9983 + 0.0000 * e([1])
(1,0) 200.0008 + 0.0000 * e([1])
 - Column 1
(0,1) 200.0008 + 0.0000 * e([1])
(1,1) -199.9983 + 0.0000 * e([1])
>

In [11]:
Th.elements[1]['indices'][0][199]

array([200,   1], dtype=uint64)

In [12]:
f = alg.sin(Th.x)
fh = alg.zeros((2,1))
fh[0,0] = f[200,0]
fh[1,0] = f[1,0]

f_eval = alg.dot_product(elLine.N,fh)


alg.gauss_integrate(f_eval,elLine.dV).real
#0.004200574877454433

0.004200574877454433

In [53]:
-np.cos(1.0)+np.cos(1-0.005)

0.004200583628662935

In [19]:
f_eval

dnumfe< nip: 4, 
(0) 0.8389472666250167
(1) 0.8412826843523152
(2) 0.8396539580252034
(3) 0.8405759929521288
>

In [22]:
0.00001249997396

1.249997396e-05

In [30]:

K

<(5, 5) sparse matrix of OTI numbers with 
         0 stored elements in LInked List format>

In [4]:
#*****************************************************************************************************
def solve_1d_problem(Th):
    
    els = Th.elements[1]

    fem.end_elements()
    
    alg = fem.get_global_algebra()
    K = alg.lil_matrix((Th.x.shape[0],Th.x.shape[0]))
    f = alg.zeros((Th.x.shape[0],1))
#     print(f)
#     print(K)
    #integrate 2D a fem array.
    for j in range(els['types'].size):

        elem = fem.element[ els['types'][j] ]

        if not elem.is_allocated():

            elem.end()
            elem.allocate(intorder=2)
            elem.allocate_spatial(1,compute_Jinv = True)

            # Here the temporals shhould be created.
            # - Elemental K
#             Kel = oti.zeros( ( elem.nbasis, elem.nbasis ) )
            # - Elemental f
#             fel = oti.zeros( ( elem.nbasis, 1 ) )
#             tmp1 = oti.zeros(nip=elem.nip)

#             tmp2 = zero()

        # end if 

        elm_nodes = els['indices'][j]

        for i in range(elm_nodes.shape[0]):
            
#             Kel.set(0.0); fel.set(0.0)

            elem.set_coordinates(Th.x,Th.y,Th.z,elm_nodes[i,:])
            elem.compute_jacobian()

            Nx = elem.Nx
            N  = elem.N
            
            Kel = alg.gauss_integrate(-alg.dot(alg.transpose(Nx),Nx)+alg.dot(alg.transpose(N),N),elem.dV)
            fel = 10*alg.gauss_integrate(alg.transpose(N),elem.dV)

            assemble_globals(Kel,fel,K,f,elm_nodes[i,:])
            
            # integral = integral + gauss_integrate( dot_product( fh, elem.N), elem.w_dJ)

            # integral = integral + oti.dot_product( fh, oti.gauss_integrate( element.N, element.w_dJ) )

        # end for

    # end for

    fem.end_elements()   
    
    # Setting Dirichlet BCs using TGV.
    TGV = 1e30

    K[0,0] = TGV
    f[0,0] = 1.0*TGV

    K[1,1] = TGV
    f[1,0] = 4.0*TGV
    
    u = alg.solve(K.tocsr(),f)

    return u,K,f

    #-----------------------------------------------------------------------------------------------------
    
def assemble_globals(Kel,fel,Kgl,fgl,elem):
    
    
    for i in range(elem.size):
        
        ii=int(elem[i])
        
        for j in range(elem.size):

            jj=int(elem[j])
        
            Kgl[ii,jj] = Kgl[ii,jj] + Kel[i,j]
            
        # end for 
        fgl[ii,0] = fgl[ii,0] + fel[i,0]
        
    # end for 
    

In [5]:
u,K,f = solve_1d_problem(Th)
print(u)

omatm1n1< shape: (201, 1), 
 - Column 0
(0,0) 1.0000 + 0.0000 * e([1])
(1,0) 4.0000 + 0.0000 * e([1])
(2,0) 0.9934 + 0.0000 * e([1])
(3,0) 0.9869 + 0.0000 * e([1])
(4,0) 0.9807 + 0.0000 * e([1])
(5,0) 0.9748 + 0.0000 * e([1])
(6,0) 0.9690 + 0.0000 * e([1])
(7,0) 0.9635 + 0.0000 * e([1])
(8,0) 0.9582 + 0.0000 * e([1])
(9,0) 0.9532 + 0.0000 * e([1])
(10,0) 0.9483 + 0.0000 * e([1])
(11,0) 0.9437 + 0.0000 * e([1])
(12,0) 0.9393 + 0.0000 * e([1])
(13,0) 0.9352 + 0.0000 * e([1])
(14,0) 0.9312 + 0.0000 * e([1])
(15,0) 0.9275 + 0.0000 * e([1])
(16,0) 0.9240 + 0.0000 * e([1])
(17,0) 0.9208 + 0.0000 * e([1])
(18,0) 0.9178 + 0.0000 * e([1])
(19,0) 0.9150 + 0.0000 * e([1])
(20,0) 0.9124 + 0.0000 * e([1])
(21,0) 0.9100 + 0.0000 * e([1])
(22,0) 0.9079 + 0.0000 * e([1])
(23,0) 0.9060 + 0.0000 * e([1])
(24,0) 0.9044 + 0.0000 * e([1])
(25,0) 0.9029 + 0.0000 * e([1])
(26,0) 0.9017 + 0.0000 * e([1])
(27,0) 0.9007 + 0.0000 * e([1])
(28,0) 0.9000 + 0.0000 * e([1])
(29,0) 0.8995 + 0.0000 * e([1])
(30,0) 0.8

In [6]:
import pyvista as pv
p = pv.BackgroundPlotter()

grid = Th.to_pv(pd=[u.real],pd_names=['u'])

p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
p.add_mesh(grid.warp_by_scalar(), show_edges=True, line_width=2,grid = True,cmap='jet',scalar_bar_args={'interactive':True})
p.show_bounds()
p.show_grid()

In [36]:
K[1,1] = K[1,1] + 3+oti.e(1)
K[1,1]

9.0000 + 3.0000 * e([1])

In [38]:
print(K)

(  1,  1) 9 + 3 * e([1])

